<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Definições-iniciais" data-toc-modified-id="Definições-iniciais-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Definições iniciais</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Configurando-random-pra-deixar-reprodutível" data-toc-modified-id="Configurando-random-pra-deixar-reprodutível-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Configurando random pra deixar reprodutível</a></span></li></ul></li><li><span><a href="#Funções-úteis" data-toc-modified-id="Funções-úteis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Funções úteis</a></span><ul class="toc-item"><li><span><a href="#Função-pra-submeter-os-resultados-e-salvar-os-arquivos-necessários-pra-replicar" data-toc-modified-id="Função-pra-submeter-os-resultados-e-salvar-os-arquivos-necessários-pra-replicar-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Função pra submeter os resultados e salvar os arquivos necessários pra replicar</a></span></li><li><span><a href="#Custom-transformers" data-toc-modified-id="Custom-transformers-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Custom transformers</a></span></li></ul></li><li><span><a href="#Carrega-dados" data-toc-modified-id="Carrega-dados-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Carrega dados</a></span></li><li><span><a href="#Criação-de-algumas-features-novas" data-toc-modified-id="Criação-de-algumas-features-novas-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Criação de algumas features novas</a></span><ul class="toc-item"><li><span><a href="#Tamanho-de-família" data-toc-modified-id="Tamanho-de-família-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Tamanho de família</a></span></li><li><span><a href="#Extrai-o-título" data-toc-modified-id="Extrai-o-título-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Extrai o título</a></span></li><li><span><a href="#Acha-aspas-ou-parênteses-no-nome" data-toc-modified-id="Acha-aspas-ou-parênteses-no-nome-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Acha aspas ou parênteses no nome</a></span></li></ul></li><li><span><a href="#Definição-do-pipeline-básico-a-ser-otimizado" data-toc-modified-id="Definição-do-pipeline-básico-a-ser-otimizado-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Definição do pipeline básico a ser otimizado</a></span><ul class="toc-item"><li><span><a href="#Tratamento-de-categóricas-(transforma-pra-pandas-tipo-categórico-e-garante-mesmas-categorias-no-treino-e-no-teste)" data-toc-modified-id="Tratamento-de-categóricas-(transforma-pra-pandas-tipo-categórico-e-garante-mesmas-categorias-no-treino-e-no-teste)-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Tratamento de categóricas (transforma pra pandas tipo categórico e garante mesmas categorias no treino e no teste)</a></span></li><li><span><a href="#Separa-as-amostras-para-treino-e-validação" data-toc-modified-id="Separa-as-amostras-para-treino-e-validação-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Separa as amostras para treino e validação</a></span></li></ul></li><li><span><a href="#Usando-lgbm-sem-interface-sklearn" data-toc-modified-id="Usando-lgbm-sem-interface-sklearn-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Usando lgbm sem interface sklearn</a></span></li></ul></div>

Vou montar aqui um pipeline mais robusto, mais próximo do produtivo, focando em lgbm

# Definições iniciais

## Imports

In [342]:
# Sklearn
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, RandomizedSearchCV, train_test_split, StratifiedKFold,cross_val_score
from sklearn.model_selection import ParameterGrid, ParameterSampler
from sklearn.metrics import roc_auc_score

# Pandas numpy etc
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

# Hyperopt
from hyperopt import fmin, hp, tpe, rand, Trials, space_eval, STATUS_OK, anneal
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
from functools import partial

# Category encoders
# from category_encoders.hashing import HashingEncoder
# from category_encoders.sum_coding import SumEncoder
# from category_encoders.woe import WOEEncoder
# from category_encoders.target_encoder import TargetEncoder
# from category_encoders.one_hot import OneHotEncoder
# from category_encoders.binary import BinaryEncoder

# Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
%matplotlib inline

# Files
# Files e utils
from tempfile import mkdtemp
from shutil import rmtree
from joblib import Memory
import joblib
import os
from tqdm import tqdm


## Configurando random pra deixar reprodutível

In [68]:
import random
random.seed(42)
np.random.seed(42)
random_global = 42

# Funções úteis

## Função pra submeter os resultados e salvar os arquivos necessários pra replicar

In [69]:
class SaveModel(object):
    def __init__(self, folder_to_save, data_train=None, data_val=None, data_test=None, model=None, str_readme=None, submission_file='submission.csv'):
        self.folder_to_save = folder_to_save
        self.data_train = data_train
        self.data_val = data_val
        self.data_test = data_test
        self.model = model
        self.str_readme = str_readme
        self.submission_file = submission_file
    
    def save_model(self):
#     Create folder if not exists:
        try:
            os.makedirs(self.folder_to_save)
        except:
            pass

    #     Salva os dados usados no treino
        if self.data_train is not None:
            joblib.dump(self.data_train, self.folder_to_save+'/train_data')


    #     Salva dados usados na validação
        if self.data_val is not None:
            joblib.dump(self.data_test, self.folder_to_save+'/validation_data')

    #     Salva dados usados no teste
        if self.data_test is not None:
            joblib.dump(self.data_test, self.folder_to_save+'/test_data')

    #     Salva modelo 
        if self.model is not None:
            joblib.dump(self.model, self.folder_to_save+'/model')   

    #     Arquivo README (é o que vai escrito pro commit)
        with open(self.folder_to_save+'/README.txt', "w") as text_file:
            text_file.write(self.str_readme)
            
#         Salva os predictions
        

    def commit_kaggle(self):
        predictions = self.model.predict(self.data_test)
        submission = pd.DataFrame({'PassengerId':self.data_test['passengerid'],'Survived':predictions})
        submission.to_csv(self.folder_to_save+'/'+self.submission_file,index=False)
#         print(f"kaggle competitions submit -c titanic -f submission.csv -m \"{self.str_readme}\"")
#         !! f"kaggle competitions submit -c titanic -f {self.folder_to_save+'/'+self.submission_file} -m \"{self.str_readme}\""
        if os.system(f"kaggle competitions submit -c titanic -f {self.folder_to_save+'/'+self.submission_file} -m \"{self.str_readme}\"") != 0:
            print('Erro submetendo o arquivo no kaggle!')
            
        print(f"kaggle competitions submit -c titanic -f {self.folder_to_save+'/'+self.submission_file} -m \"{self.str_readme}\"")
    
        

## Custom transformers

In [70]:
from sklearn.base import BaseEstimator, TransformerMixin
from functools import reduce

class TransformToDF(BaseEstimator, TransformerMixin):
    '''Wrapper para usar transformers do sklearn mas retorna um dataframe Pandas. Projetei para usar com transformers que não
    mudam o número de colunas na saída.'''
    
    def __init__(self, sklearn_transformer, return_df=True):
        self.sklearn_transformer = sklearn_transformer
        self.return_df = return_df
    
    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            self.col_names = X.columns.values.tolist()
            
        self.sklearn_transformer.fit(X, y=None)
        return self
        
    def transform(self, X, y=None):
    # assumes X is a DataFrame
        if self.return_df:
            return pd.DataFrame(self.sklearn_transformer.transform(X[self.col_names]), index=X.index, columns=self.col_names)
        else:
            return self.sklearn_transformer.transform(X)
        
class DFFeatureUnion(BaseEstimator,TransformerMixin):
    # FeatureUnion but for pandas DataFrames

    def __init__(self, transformer_list):
        self.transformer_list = transformer_list

    def fit(self, X, y=None):
        for (name, t) in self.transformer_list:
            t.fit(X, y)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xts = [t.transform(X) for _, t in self.transformer_list]
        Xunion = reduce(lambda X1, X2: pd.merge(X1, X2, left_index=True, right_index=True), Xts)
        return Xunion
    
class ColumnExtractor(BaseEstimator,TransformerMixin):

    def __init__(self, cols, return_df=True):
        self.cols = cols
        self.return_df = return_df

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        if self.return_df:
            return X[self.cols]
        else:
            return X[self.cols].values

# Carrega dados


In [71]:
df_train_val = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
df_train_val.columns = [x.lower() for x in df_train_val.columns]
df_test.columns = [x.lower() for x in df_test.columns]

print(f'Print shape df_train_val: {df_train_val.shape}')
print(f'Print shape df_test: {df_test.shape}')


Print shape df_train_val: (891, 12)
Print shape df_test: (418, 11)


# Criação de algumas features novas

## Tamanho de família

In [72]:
full_data = [df_train_val, df_test]
for dataset in full_data:
    dataset['family_size'] = 1 + dataset['parch'] + dataset['sibsp']
    if 'survived' in dataset.columns:
        display(dataset.fillna(-999).groupby('family_size')['survived'].agg(['size', 'mean']).reset_index())
    else:
        display(dataset.fillna(-999).groupby('family_size').size())

,family_size,size,mean
0,1,537,0.303538
1,2,161,0.552795
2,3,102,0.578431
3,4,29,0.724138
4,5,15,0.200000
5,6,22,0.136364
6,7,12,0.333333
7,8,6,0.000000
8,11,7,0.000000


family_size
1     253
2      74
3      57
4      14
5       7
6       3
7       4
8       2
11      4
dtype: int64

Aparentemente depois de 4 familiares, talvez seja bom juntar depois

## Extrai o título

In [73]:
full_data = [df_train_val, df_test]
for dataset in full_data:
    dataset['title'] = dataset['name'].str.findall('([A-Z][a-z]+)\.').map(lambda x: x[0])
    if 'survived' in dataset.columns:
        display(dataset.fillna(-999).groupby('title')['survived'].agg(['size', 'mean']).reset_index().sort_values(by='mean', ascending=False))
    else:
        display(dataset.fillna(-999).groupby('title').size())
        

,title,size,mean
16,Sir,1,1.000000
2,Countess,1,1.000000
14,Ms,1,1.000000
11,Mme,1,1.000000
6,Lady,1,1.000000
10,Mlle,2,1.000000
13,Mrs,125,0.792000
9,Miss,182,0.697802
8,Master,40,0.575000
1,Col,2,0.500000


title
Col         2
Dona        1
Dr          1
Master     21
Miss       78
Mr        240
Mrs        72
Ms          1
Rev         2
dtype: int64

## Acha aspas ou parênteses no nome

In [74]:
full_data = [df_train_val, df_test]
for dataset in full_data:
    dataset['name_aspas'] = dataset['name'].str.findall('\"').map(lambda x: len(x) >= 1 ).astype('int')
    dataset['name_parenteses'] = dataset['name'].str.findall('\(').map(lambda x: len(x) >= 1 ).astype('int')
    if 'survived' in dataset.columns:
        display(dataset.fillna(-999).groupby('name_aspas')['survived'].agg(['size', 'mean']).reset_index().sort_values(by='mean', ascending=False))
        display(dataset.fillna(-999).groupby('name_parenteses')['survived'].agg(['size', 'mean']).reset_index().sort_values(by='mean', ascending=False))
    else:
        display(dataset.fillna(-999).groupby('name_aspas').size())
        display(dataset.fillna(-999).groupby('name_parenteses').size())
        

,name_aspas,size,mean
1,1,53,0.716981
0,0,838,0.362768


,name_parenteses,size,mean
1,1,143,0.769231
0,0,748,0.310160


name_aspas
0    396
1     22
dtype: int64

name_parenteses
0    340
1     78
dtype: int64

# Definição do pipeline básico a ser otimizado

## Tratamento de categóricas (transforma pra pandas tipo categórico e garante mesmas categorias no treino e no teste)

In [75]:
num_cols = ['age', 'sibsp', 'parch', 'fare', 'family_size']
cat_cols = ['sex', 'embarked', 'title', 'name_aspas', 'name_parenteses', 'pclass']

In [76]:
dict_dtypes = {}
for col in cat_cols + num_cols:
    if col in cat_cols:
        dict_dtypes[col] = CategoricalDtype(categories=df_train_val[col].dropna().unique(), ordered=False)
    else:
        dict_dtypes[col] = df_train_val[col].dtypes

In [77]:
df_train_val = df_train_val.astype(dict_dtypes)
df_test = df_test.astype(dict_dtypes)

In [97]:
df_train_val[cat_cols].apply(lambda x: x.cat.codes)

,sex,embarked,title,name_aspas,name_parenteses,pclass
0,0,0,0,0,0,0
1,1,1,1,0,1,1
2,1,0,2,0,0,0
3,1,0,1,0,1,1
4,0,0,0,0,0,0
5,0,2,0,0,0,0
6,0,0,0,0,0,1
7,0,0,3,0,0,0
8,1,0,1,0,1,0
9,1,1,1,0,1,2


## Separa as amostras para treino e validação

In [421]:
df_train, df_val = train_test_split(df_train_val, test_size = 100, shuffle=True, stratify=df_train_val['survived'], random_state=random_global)
print(f'Print shape df_train: {df_train.shape}')
print(f'Print shape df_val: {df_val.shape}')

Print shape df_train: (791, 16)
Print shape df_val: (100, 16)


In [85]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_global)

In [610]:
def get_cat_codes(X, return_df=False, return_as_category=False):
    if return_df:
        if return_as_category:
            return X.apply(lambda x: x.cat.codes).astype('category')
        else:
            return X.apply(lambda x: x.cat.codes)
    else:
        return X.apply(lambda x: x.cat.codes).values
get_cat_codes(df_train_val[cat_cols], return_df=True, return_as_category=True).describe()

,sex,embarked,title,name_aspas,name_parenteses,pclass
count,891,891,891,891,891,891
unique,2,4,17,2,2,3
top,0,0,0,0,0,0
freq,577,644,517,838,748,491


In [529]:
make_pipeline(FunctionTransformer(get_cat_codes, validate=False, kw_args={'return_df':True, 'return_as_category':True}))\
.fit_transform(df_train_val[cat_cols]).dtypes

sex                category
embarked           category
title              category
name_aspas         category
name_parenteses    category
pclass             category
dtype: object

In [598]:
make_pipeline(ColumnExtractor(num_cols, return_df=False), TransformToDF(SimpleImputer(fill_value=0, strategy='constant')\
                                                                        ,return_df=False)\
              ,TransformToDF(StandardScaler(), return_df=False))\
.fit_transform(df_train_val[num_cols])

array([[-0.10231279,  0.43279337, -0.47367361, -0.50244517,  0.05915988],
       [ 0.80749164,  0.43279337, -0.47367361,  0.78684529,  0.05915988],
       [ 0.12513832, -0.4745452 , -0.47367361, -0.48885426, -0.56097483],
       ...,
       [-1.35329389,  0.43279337,  2.00893337, -0.17626324,  1.29942929],
       [ 0.12513832, -0.4745452 , -0.47367361, -0.04438104, -0.56097483],
       [ 0.46631498, -0.4745452 , -0.47367361, -0.49237783, -0.56097483]])

In [643]:
from lightgbm import LGBMClassifier
def f_wrap_space_eval(hp_space, trial):
    """
    Utility function for more consise optimization history extraction
    
    Parameters:
    ----------------
    hp_space : hyperspace from which points are sampled
    trial : hyperopt.Trials object
    
    Returns:
    ----------------
    : dict(
        k: v
    ), where k - label of hyperparameter, v - value of hyperparameter in trial
    """
    
    return space_eval(hp_space, {k: v[0] for (k, v) in trial['misc']['vals'].items() if len(v) > 0})


def f_unpack_dict(dct):
    """
    Unpacks all sub-dictionaries in given dictionary recursively. There should be no duplicated keys 
    across all nested subdictionaries, or some instances will be lost without warning
    
    Parameters:
    ----------------
    dct : dictionary to unpack
    
    Returns:
    ----------------
    : unpacked dictionary
    """
    
    res = {}
    for (k, v) in dct.items():
        if isinstance(v, dict):
            res = {**res, **f_unpack_dict(v)}
        else:
            res[k] = v
            
    return res

# Função a ser minimizada
def f_to_min1(hps, X, y, cv):
    """
    Target function for optimization
    
    Parameters:
    ----------------
    hps : sample point from search space
    X : feature matrix
    y : target array
    ncv : number of folds for cross-validation
    
    Returns:
    ----------------
    : target function value (negative mean cross-val ROC-AUC score)
    """
    

    model = get_pipeline(hps)
    crossval = cross_validate(model, X, y, return_train_score=True, cv=cv, scoring='roc_auc')

    return {'loss': -crossval['test_score'].mean(), 'status': STATUS_OK}


hp_space_lgb ={
    'num_missing_strategy': hp.choice('num_missing_strategy', ['mean', 'median', 'constant']),
#     'categ_encoder': hp.choice('categ_encoder', ['woe','one_hot','target_encoder', 'lgbm_internal']),
    'categ_encoder': hp.choice('categ_encoder', ['one_hot','label_encoder', 'lgbm_internal']),
        # type refers to classifier type: either logit or SVM
    'clf_type': hp.choice('clf_type', [
        {
            'type': 'logit',
            'clf': {
                'C': hp.loguniform('logit.C', -4.0*np.log(10.0), 4.0*np.log(10.0)), 
                'class_weight': hp.choice('logit.class_weight', [None, 'balanced'])
            }
        }, 
        {
            'type': 'SVM', 
            'clf': {
                'C': hp.loguniform('svm.C', -4.0*np.log(10.0), 4.0*np.log(10.0)), 
                'class_weight': hp.choice('svm.class_weight', [None, 'balanced']), 
                'kernel': 'rbf', 
                'gamma': hp.choice('svm.gamma', ['auto', 'scale'])
            }
        },
        {
            'type': 'lgbm_rf',
            'clf': {
                'max_depth': ho_scope.int(hp.uniform('lgbm_rf.max_depth', low=1, high=11)),
                'boosting_type': "rf",
                'num_leaves': 165,
                'colsample_bytree': .5,
                'n_estimators': 2000,
                'min_child_weight': 0,
                'min_child_samples':0,
                'subsample': .632, # Standard RF bagging fraction
                'subsample_freq': 1,
                'min_split_gain': 0,
                'reg_alpha': 0, 
                'reg_lambda': 0,
                'subsample_for_bin':200000,
                'min_data_per_group': 5
            }
        },
        {
            'type': 'RandomForest', 
            'clf': {
                    'bootstrap': hp.choice('rf.bootstrap', [False, True]),
                    'class_weight': hp.choice('rf.class_weight',['balanced', 'balanced_subsample', None]),
                    'criterion': hp.choice('rf.criterion', ['gini', 'entropy']),
                    'max_depth': ho_scope.int(hp.uniform('rf.max_depth', low=1, high=11)),
                    'max_features': hp.choice('rf.max_features', ['auto','sqrt', None]),
                    'max_leaf_nodes': ho_scope.int(hp.uniform('rf.max_leaf_nodes', low=2, high=1024)),
                    'min_samples_split': ho_scope.int(hp.uniform('rf.min_samples_split', low=2, high=1024)),
                    'min_weight_fraction_leaf': 0.1 * ho_scope.int(hp.uniform('rf.min_weight_fraction_leaf', low=0, high=5)),
                    'n_estimators': 100 * ho_scope.int(hp.uniform('rf.n_estimators', low=1, high=10)),
                    'oob_score': False
            }
        }
    ])
}


def get_pipeline(hps, use_df=True):
    """
    Constructs estimator
    
    Parameters:
    ----------------
    hps : sample point from search space
    use_df: if Pandas Dataframes should be used. 
    Can reduce speed but maybe necessary for interpretability or other reasons.
    
    Returns:
    ----------------
    model : sklearn.Pipeline.pipeline with hyperparameters set up as per hps
    """
    
#     Escolha de encoder
    if hps['categ_encoder'] == 'one_hot':
        kwargs_get_cat_codes={'return_df':use_df, 'return_as_category':False}
        categ_encoder = OneHotEncoder(categories='auto', sparse=False)
    elif hps['categ_encoder'] == 'label_encoder':
        kwargs_get_cat_codes={'return_df':use_df, 'return_as_category':False}
        categ_encoder = None
#     'lgbm_internal' só vai fazer diferença pro lgbm, pros restantes vai ser idêntico a um label_encoder
    # Por enquanto aqui vai precisar usar pandas se quiser usar as features categóricas no lgbm
    elif hps['categ_encoder'] == 'lgbm_internal':
        kwargs_get_cat_codes={'return_df':use_df, 'return_as_category':True}
        categ_encoder = None
    
    cat_pipeline = make_pipeline(ColumnExtractor(cat_cols, return_df=use_df),\
                                 FunctionTransformer(get_cat_codes, validate=False, kw_args=kwargs_get_cat_codes))
    
#     Para logística precisaremos normalizar as features
    if hps['clf_type']['type'] == 'logit':
        num_pipeline = make_pipeline(ColumnExtractor(num_cols, return_df=use_df),\
                                     TransformToDF(SimpleImputer(fill_value=0, strategy=hps['num_missing_strategy']),\
                                                   return_df=use_df),TransformToDF(StandardScaler(), return_df=use_df))
    else:
        num_pipeline = make_pipeline(ColumnExtractor(num_cols, return_df=use_df),\
                                     TransformToDF(SimpleImputer(fill_value=0, strategy=hps['num_missing_strategy']),\
                                                   return_df=use_df))
    
    if hps['clf_type']['type'] == 'logit':
        clf = LogisticRegression(**hps['clf_type']['clf'], solver='sag', max_iter=25000, random_state=random_global)
    elif hps['clf_type']['type'] == 'SVM':
        clf = SVC(**f_unpack_dict(hps['clf_type']['clf']), probability=True, random_state=random_global)
    elif hps['clf_type']['type'] == 'RandomForest':
        clf = RandomForestClassifier(**f_unpack_dict(hps['clf_type']['clf']), random_state=random_global)
    elif hps['clf_type']['type'] == 'lgbm_rf':
        clf = LGBMClassifier(**f_unpack_dict(hps['clf_type']['clf']), random_state=random_global)
    else:
        raise KeyError('Unknown classifier type hyperparameter value: {0}'.format(hps['clf_type']['type']))
    
    
    model = Pipeline([('features', DFFeatureUnion([('cat_flow',cat_pipeline), ('num_flow',num_pipeline)])), ('clf', clf)])
    return model

In [644]:
trials_clf1 = Trials()

best_clf1=fmin(partial(f_to_min1,X=df_train, y=df_train['survived'], cv=kf)
     ,hp_space_lgb, algo=tpe.suggest, max_evals=3, trials=trials_clf1, rstate=np.random.RandomState(random_global))

100%|████████████████████████████████████████████████████| 3/3 [00:49<00:00, 12.85s/it, best loss: -0.8861751508107044]


In [647]:
space_eval(hp_space_lgb, best_clf1)

{'categ_encoder': 'label_encoder',
 'clf_type': {'clf': {'boosting_type': 'rf',
   'colsample_bytree': 0.5,
   'max_depth': 10,
   'min_child_samples': 0,
   'min_child_weight': 0,
   'min_data_per_group': 5,
   'min_split_gain': 0,
   'n_estimators': 2000,
   'num_leaves': 165,
   'reg_alpha': 0,
   'reg_lambda': 0,
   'subsample': 0.632,
   'subsample_for_bin': 200000,
   'subsample_freq': 1},
  'type': 'lgbm_rf'},
 'num_missing_strategy': 'median'}

In [648]:
get_pipeline(space_eval(hp_space_lgb, best_clf1))

Pipeline(memory=None,
         steps=[('features',
                 DFFeatureUnion(transformer_list=[('cat_flow',
                                                   Pipeline(memory=None,
                                                            steps=[('columnextractor',
                                                                    ColumnExtractor(cols=['sex',
                                                                                          'embarked',
                                                                                          'title',
                                                                                          'name_aspas',
                                                                                          'name_parenteses',
                                                                                          'pclass'],
                                                                                    return_df=True)),
            

# Usando lgbm sem interface sklearn

In [416]:
import lightgbm as lgb

In [510]:
lgb_train = lgb.Dataset(df_train[cat_cols + num_cols], df_train['survived'])
lgb_eval = lgb.Dataset(df_val[cat_cols + num_cols], df_val['survived'], reference=lgb_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

params = {
    'max_depth': -1,
     'boosting_type': 'rf',
     'num_leaves': 64,
     'colsample_bytree': 0.5,
     'n_estimators': 100,
     'min_child_weight': 5,
     'min_child_samples': 10,
     'subsample': 0.632,
     'subsample_freq': 1,
     'min_split_gain': 0,
     'reg_alpha': 0,
     'reg_lambda': 0,
     'subsample_for_bin': 100,
     'n_jobs': -1,
    'metric': {'auc'},
    'verbose':10
}
print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                valid_sets=[lgb_train,lgb_eval],
                valid_names=['train', 'eval'],
#                 early_stopping_rounds=0
               )


Starting training...


C:\Users\T-Gamer\Anaconda3\envs\sklearn_novo\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\T-Gamer\Anaconda3\envs\sklearn_novo\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	train's auc: 0.88935	eval's auc: 0.819397
[2]	train's auc: 0.906757	eval's auc: 0.822368
[3]	train's auc: 0.900694	eval's auc: 0.837012
[4]	train's auc: 0.906952	eval's auc: 0.837861
[5]	train's auc: 0.906426	eval's auc: 0.837649
[6]	train's auc: 0.907709	eval's auc: 0.836375
[7]	train's auc: 0.907969	eval's auc: 0.843591
[8]	train's auc: 0.911319	eval's auc: 0.844864
[9]	train's auc: 0.910603	eval's auc: 0.842742
[10]	train's auc: 0.911269	eval's auc: 0.841469
[11]	train's auc: 0.910472	eval's auc: 0.838922
[12]	train's auc: 0.911231	eval's auc: 0.832131
[13]	train's auc: 0.912731	eval's auc: 0.840195
[14]	train's auc: 0.912204	eval's auc: 0.841044
[15]	train's auc: 0.913322	eval's auc: 0.835102
[16]	train's auc: 0.912815	eval's auc: 0.841469
[17]	train's auc: 0.914524	eval's auc: 0.844864
[18]	train's auc: 0.913423	eval's auc: 0.84444
[19]	train's auc: 0.913933	eval's auc: 0.849958
[20]	train's auc: 0.91394	eval's auc: 0.847835
[21]	train's auc: 0.914271	eval's auc: 0.849958
[22]